In [1]:
import asyncio

In [2]:
class ChatServer:
    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.clients = []
        self.rooms = {}

    async def handle_client(self, reader, writer):
        client_name = await reader.read(100)
        client_name = client_name.decode().strip()
        print(f"New client connected: {client_name}")

        writer.write(b"Connected to the chat server\n")
        await writer.drain()

        self.clients.append((reader, writer))

        while True:
            data = await reader.read(100)
            message = data.decode().strip()

            if message == "quit":
                break

            if message.startswith("join"):
                room = message.split()[1]
                self.join_room(client_name, room)
            elif message.startswith("leave"):
                room = message.split()[1]
                self.leave_room(client_name, room)
            else:
                await self.send_message(client_name, message)

        print(f"Client disconnected: {client_name}")
        self.clients.remove((reader, writer))
        writer.close()
        await writer.wait_closed()

    def join_room(self, client_name, room):
        if room not in self.rooms:
            self.rooms[room] = []
        self.rooms[room].append(client_name)
        print(f"{client_name} joined room {room}")

    def leave_room(self, client_name, room):
        if room in self.rooms and client_name in self.rooms[room]:
            self.rooms[room].remove(client_name)
            print(f"{client_name} left room {room}")

    async def send_message(self, sender, message):
        for room in self.rooms.values():
            if sender in room:
                recipients = [client for client in self.clients if client[1] != sender and client[1] in room]
                for recipient in recipients:
                    recipient[1].write(f"{sender}: {message}\n".encode())
                    await recipient[1].drain()

    async def start(self):
        server = await asyncio.start_server(self.handle_client, self.host, self.port)

        addr = server.sockets[0].getsockname()
        print(f"Server started on {addr}")

        async with server:
            await server.serve_forever()

In [4]:
if __name__ == "__main__":
    chat_server = ChatServer("localhost", 8888)
    #asyncio.run(chat_server.start())